In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
arquivoDrive = "/content/drive/MyDrive/Grupo1 - Estrutura de Dados/Json/hotel_reservations.json"

In [ ]:
import json
from collections import deque

# ----------------------------
# Exercício 1 — Lista
# ----------------------------
def carregar_reservas(caminho: str) -> list[dict]:
    """Carrega o JSON em uma lista de dicionários e retorna o catálogo de reservas."""
    print("Função carregar_reservas chamada")
    # implementar leitura do arquivo JSON

    with open(caminho, 'r') as arquivo:
      dados = json.load(arquivo)
    espera = deque()
    for r in dados:
      ano = r.get("arrival_year")
      mes = r.get("arrival_month")
      dia = r.get("arrival_date")
      r["arrival_tuble"] = (ano, mes, dia)
      espera.append(r)
    print(f"> {len(dados)} reservas carregadas")
    return espera

# ----------------------------
# Exercício 2 — Fila
# ----------------------------
def montar_fila(reservas: list[dict], ano: int, mes: int, dia: int, estrutura="deque"):
    """Monta a fila de check-in com base nas reservas do dia (Not_Canceled).
       estrutura pode ser 'queue' ou 'deque'."""
    print("Função montar_fila chamada")
    # implementar criação da fila
    reservas_do_dia = deque()
    for r in reservas:
      if (r.get("arrival_year") == ano and
          r.get("arrival_month") == mes and
          r.get("arrival_day") == dia and
          r.get("booking_status") == "Not_Canceled"):
        reservas_do_dia.append(r) #FIFO
    print(f"> {len(reservas_do_dia)} reservas para {dia:02d}/{mes:02d}/{ano:04d} (Nao Canceladas)")
    return reservas_do_dia

# ----------------------------
# Exercício 3 — Pilha (Undo)
# ----------------------------
def registrar_acao(pilha: list, reserva: dict):
    """Adiciona na pilha o check-in realizado."""
    print("Função registrar_acao chamada")
    # implementar push na pilha
    if reserva is None:
        print("> nada a registrar")
        return
    pilha.append(reserva)  # push (topo à direita)
    print(f"> push OK (checkin tamanho={len(pilha)})")

def desfazer_acao(pilha: list) -> dict:
    """Remove o último check-in da pilha e retorna a reserva desfeita."""
    print("Função desfazer_acao chamada")
    # implementar pop da pilha
    if not pilha:
        print("> pilha vazia")
        return None
    r = pilha.pop()
    print(f"> pop OK (checkin tamanho={len(pilha)})")
    return r

# ----------------------------
# Exercício 4 — Interação Fila ↔ Pilha
#   - 'chamar_proximo': move FIFO -> LIFO   (reservas_do_dia -> checkin)
#   - 'devolver_para_fila': desfaz LIFO -> FIFO (checkin -> reservas_do_dia no INÍCIO)
# ----------------------------
def chamar_proximo(reservas_do_dia: "deque[dict]", checkin: "deque[dict]") -> dict:
    """Atende o próximo da FILA (FIFO) e registra na PILHA (LIFO)."""
    print("Função chamar_proximo chamada")
    if not reservas_do_dia:
        print("> fila do dia vazia")
        return None
    r = reservas_do_dia.popleft()     # FIFO
    registrar_acao(checkin, r)        # LIFO (push)
    print(f"> check-in realizado: {r.get('Booking_ID')}")
    return r

def devolver_para_fila(checkin: "deque[dict]", reservas_do_dia: "deque[dict]"):
    """
    Desfaz o último check-in: tira da PILHA (pop) e coloca
    no INÍCIO da FILA do dia (appendleft), garantindo prioridade.
    """
    print("Função devolver_para_fila chamada")
    r = desfazer_acao(checkin)        # LIFO (pop)
    if r is None:
        return
    reservas_do_dia.appendleft(r)     # volta pro começo da FILA (FIFO)
    print(f"> devolvido à fila do dia (início): {r.get('Booking_ID')}")

# ----------------------------
# Exercício 5 — Relatório Integrado
# ----------------------------
def gerar_relatorio(reservas: list[dict], fila, pilha: list, escolha: int):
    """Gera relatório com dados da lista, fila e pilha.
       escolha define a métrica extra a ser exibida:
       1 = média preço por quarto
       2 = top 3 tipos de quarto
       3 = taxa de cancelamento"""
    print("Função gerar_relatorio chamada")
    # implementar relatório
    print("\n=== RELATÓRIO INTEGRADO ===")
    print(f"- Total na LISTA (reservas): {len(reservas)}")
    print(f"- Tamanho da FILA: {len(fila) if fila is not None else 0}")
    print(f"- Tamanho da PILHA: {len(pilha)}")

    if escolha == 1:
        valores = []
        for r in reservas:
            v = r.get("avg_price_per_room")
            if isinstance(v, (int, float)):
                valores.append(float(v))
            else:
                try:
                    valores.append(float(v))
                except:
                    pass
        if valores:
            media = sum(valores) / len(valores)
            print(f"- (1) Média preço por quarto: {media:.2f}")
        else:
            print("- (1) Sem valores para média.")

    elif escolha == 2:
        cont = {}
        for r in reservas:
            rt = r.get("room_type_reserved")
            if rt:
                cont[rt] = cont.get(rt, 0) + 1
        top = sorted(cont.items(), key=lambda x: x[1], reverse=True)[:3]
        print("- (2) Top 3 tipos de quarto:")
        if top:
            for i, (rt, c) in enumerate(top, 1):
                print(f"  {i}. {rt}: {c}")
        else:
            print("  sem dados.")

    elif escolha == 3:
        cancel = sum(1 for r in reservas if r.get("booking_status") == "Canceled")
        ok     = sum(1 for r in reservas if r.get("booking_status") == "Not_Canceled")
        tot = cancel + ok
        if tot:
            tx = 100.0 * cancel / tot
            print(f"- (3) Taxa de cancelamento: {tx:.2f}% (Canceladas={cancel}, Não canceladas={ok})")
        else:
            print("- (3) Sem status suficientes.")
    else:
        print("- Escolha inválida (1/2/3).")

    if fila:
        preview = [r.get("Booking_ID") for r in list(fila)[:5]]
        print(f"- Próximos na fila (até 5): {preview}")
    print("=================================\n")

In [ ]:
from IPython.display import clear_output
from collections import deque

# Estruturas VIVAS do programa (mantém as que você já tem)
espera = deque()
reservas_do_dia = deque()
checkin = deque()

def pause():
    """Pausa amigável: Enter volta ao menu; se interromper, só continua."""
    try:
        input("\n[Enter] para voltar ao menu...")
    except (KeyboardInterrupt, EOFError):
        pass

def safe_input(prompt: str):
    """Lê uma string do usuário com proteção contra interromper/parar."""
    try:
        return input(prompt)
    except (KeyboardInterrupt, EOFError):
        return None

while True:
    clear_output(wait=True)
    print("\n=== HOTEL RESERVATIONS — MENU ===")
    print("1) Carregar base na ESPERA (deque viva)")
    print("2) Montar FILA do dia (reservas não canceladas)")
    print("3) Fazer check-in (mover FILA -> PILHA)")
    print("4) Desfazer último check-in (mover PILHA -> FILA, início)")
    print("5) Relatório integrado (1/2/3)")
    print("0) Sair")

    ans = safe_input("Escolha uma opção (0–5): ")
    if ans is None:
        print("\nInterrompido. Voltando ao menu…")
        continue
    op = ans.strip()

    if op == "1":
        caminho = arquivoDrive
        if not caminho:
            print("Nome vazio/abortado.")
            pause(); continue
        espera = carregar_reservas(caminho.strip())
        reservas_do_dia.clear(); checkin.clear()
        pause()

    elif op == "2":
        if not espera:
            print("Carregue a base primeiro (opção 1)."); pause(); continue
        a = safe_input("Ano: "); m = safe_input("Mês: "); d = safe_input("Dia: ")
        try:
            ano, mes, dia = int(a), int(m), int(d)
        except (TypeError, ValueError):
            print("Valores inválidos."); pause(); continue
        reservas_do_dia = montar_fila(espera, ano, mes, dia, "deque")
        print(f"Fila do dia montada com {len(reservas_do_dia)} reservas.")
        pause()

    elif op == "3":
        if not reservas_do_dia:
            print("Monte a FILA do dia primeiro (opção 2)."); pause(); continue
        qtd = safe_input("Quantos check-ins agora? [padrão=1]: ")
        try:
            n = 1 if not qtd else max(1, int(qtd))
        except ValueError:
            n = 1
        for _ in range(n):
            r = chamar_proximo(reservas_do_dia, checkin)
            if r is None: break
        pause()

    elif op == "4":
        if not checkin:
            print("Nenhum check-in para desfazer."); pause(); continue
        devolver_para_fila(checkin, reservas_do_dia)  # volta para o INÍCIO
        pause()

    elif op == "5":
        if not espera:
            print("Carregue a base primeiro (opção 1)."); pause(); continue
        print("Métrica:\n 1) Média preço por quarto\n 2) Top 3 tipos de quarto\n 3) Taxa de cancelamento")
        e = safe_input("Digite 1, 2 ou 3: ")
        try:
            escolha = int(e) if e is not None else 0
        except ValueError:
            escolha = 0
        gerar_relatorio(espera, reservas_do_dia, checkin, escolha)
        pause()

    elif op == "0":
        print("Encerrando programa."); break
    else:
        print("Opção inválida."); pause()



=== HOTEL RESERVATIONS — MENU ===
1) Carregar base na ESPERA (deque viva)
2) Montar FILA do dia (reservas não canceladas)
3) Fazer check-in (mover FILA -> PILHA)
4) Desfazer último check-in (mover PILHA -> FILA, início)
5) Relatório integrado (1/2/3)
0) Sair
